### This Jupyter notebook will be used to determine the user defines variables

#### Step 1 - Retrieve Location of Desired Study Area
Using either a shapefile, coordinate, or WKT the study area for where satalite imagery will be downloaded is to be determined

In [5]:
#import requirements
import pandas as pd
import geopandas as gpd
import asf_search as asf
from shapely import wkt

### Find data using a geographic search
To begin, we will need to craft a suitable WKT. ASF's [Vertex](https://search.asf.alaska.edu) can be helpful in this regard, as it allows you to draw on a map, or import a geospatial file such as a shapefile or geojson, after which a WKT string can be copied and used elsewhere. <br><br>
This method is good for bulk downloads of all scenes matching the description, excercise with caution as it will download the max limit (currently set at 10) and can take awhile.

In [6]:
#Select location using a WKT obtained from ASF's Vertex search site or copied from a geodatabase (must resemble: 'POLYGON((-85.5523 64.8284,-84.9823 64.8284,-84.9823 65.3214,-85.5523 65.3214,-85.5523 64.8284))')

location = input('input WKT string')
print('The given location is: ' + location)

The given location is: POLYGON((-85.5523 64.8284,-84.9823 64.8284,-84.9823 65.3214,-85.5523 65.3214,-85.5523 64.8284))


In [7]:
#input date range for search
dateStart = input('input start date in YYYY-MM-DD')
dateEnd = input('input end date in YYYY-MM-DD')

print('The date range of the search will be: ' + dateStart + ' to ' + dateEnd)

The date range of the search will be: 2020-01-01 to 2022-01-05


In [19]:
#perform the search for results with the given locations centroid and date range

centroidwkt = wkt.loads(location)
centroid = centroidwkt.centroid.wkt


centroid_results = asf.geo_search(
        intersectsWith=centroid,
        platform=asf.PLATFORM.SENTINEL1,
        maxResults=10,
        start=dateStart,
        end=dateEnd)

print(f'{len(centroid_results)} results found')

In [18]:
#perform the search for results with the given location and date range, use if centroid yields no results

geo_results = asf.geo_search(
        intersectsWith=location,
        platform=asf.PLATFORM.SENTINEL1,
        maxResults=10,
        start=dateStart,
        end=dateEnd)

print(f'{len(geo_results)} results found')

10 results found


### Finding data using an existing scene name
If you already have the name of the scene (i.e. 'S1A_EW_GRDM_1SDH_20150205T122016_20150205T122116_004488_005812_F6B0') you can use this part of the program. This is the recommended method as it allows the user to visually verify the scene before downloading.

In [10]:
#add scenes to the list to search using ASF's Vertex online app 
scene_list = []
scene_list.append(input('input the scene here (i.e. \'S1A_EW_GRDM_1SDH_20150205T122016_20150205T122116_004488_005812_F6B0'))


In [11]:
granule_results = asf.granule_search(scene_list)

print(f'{len(granule_results)} results found')

2 results found


#### Login to downlaod data

You to have an [Earthdata Login](https://urs.earthdata.nasa.gov/) account to access the download function. The easiest way to check that your EDL account is in order is to simply go to [Vertex](https://search.asf.alaska.edu) and try to download a product.

In [12]:
import getpass
from pathlib import Path

dirs = ['downloads']

In [13]:
#login to ASF using EE credentials
username = input('Username:')
password = getpass.getpass('Password:')

try:
    user_pass_session = asf.ASFSession().auth_with_creds(username, password)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

    #in the future i may add token and cookie login features

Success!


#### Download the data
The scene search will download the specific file the user found in [Vertex](https://search.asf.alaska.edu), this may be prefered to geographic search.<br><br>
The geographic search methods are good for bulk downloads of all scenes matching the description, excercise with caution as it will download the max limit (currently set at 10) and can take awhile.

In [14]:
import os.path
from os import mkdir

In [15]:
#set the downlaod directory, or create it if it doesnt exist
downPath = './downloads'
isDir = os.path.isdir(downPath)

if isDir == False:
    mkdir('downloads')

In [ ]:
#download results from search
from os import listdir

#have user choose which result type to download
resultChoice=input('Type either: centroid, geo, or scene depending on the search criteria used')

if resultChoice == 'centroid':
    results = centroid_results
if resultChoice == 'geo':
    results = geo_results
if resultChoice == 'scene':
    results = granule_results
# else:
#     print('error, please try again')
    

results.download(path='./downloads', session=user_pass_session)

listdir('./downloads')